In [21]:
from nuplan.database.nuplan_db_orm.nuplandb import NuPlanDB
from pathlib import Path
from nuplan.database.nuplan_db_orm.nuplandb_wrapper import NuPlanDBWrapper
from nuplan.database.nuplan_db_orm.frame import Frame
import os

In [22]:
db_path = "/cailiu2/Diffusion-Planner/data/2021.10.21.14.43.30_veh-28_01244_01519.db"
data_root = "/cailiu2/Diffusion-Planner/data/"
map_path = "/cailiu2/Diffusion-Planner/data/maps"
map_version = "nuplan-maps-v1.0"

In [23]:
#使用wrapper 加载数据
nuplan_db_wrapper = NuPlanDBWrapper(    map_root=map_path,
    db_files=[db_path],  # 需要是列表
    map_version=map_version,
    data_root = data_root)

In [24]:
log_db_name = nuplan_db_wrapper.log_names
print(log_db_name[0])
log_db = nuplan_db_wrapper.get_log_db(log_db_name[0])

2021.10.21.14.43.30_veh-28_01244_01519


In [25]:
lidar_pcs = log_db.lidar_pc
print(f"The number of lidar_pcs in this log file is {len(lidar_pcs)}.")

The number of lidar_pcs in this log file is 5500.


In [26]:
#一种最简单的方法：通过外键链接查询对象的属性，如果数据规模较大时，会非常慢
example_lidar_pc = lidar_pcs[0]
example_ego_pos = example_lidar_pc.ego_pose
print(f"Lidar_pc token: {example_lidar_pc.token}.")
print(f"Ego pose: <{example_ego_pos.x}, {example_ego_pos.y}, {example_ego_pos.z}>.")

Lidar_pc token: 558a7a2075925da2.
Ego pose: <331745.4093055056, 4690691.498171177, -4.570515843069268>.


In [27]:
#这是一个通过sql session查询对应类型表的方法
from nuplan.database.nuplan_db_orm.lidar_pc import LidarPc

lidar_pc_objs = log_db.session.query(LidarPc) \
  .filter(LidarPc.timestamp > 1620857890400393) \
  .order_by(LidarPc.timestamp) \
  .limit(10) \
  .all()

print(f"Number of lidar_pcs returned: {len(lidar_pc_objs)}.")

Number of lidar_pcs returned: 10.


In [32]:
from nuplan.database.nuplan_db.query_session import execute_one, execute_many

query = """
SELECT COUNT(*) AS cnt
FROM lidar_pc;
"""
print(f"{log_db_name[0]}.db")
result = execute_one(query, (), os.path.join(data_root, f"{log_db_name[0]}.db"))
print(f"The number of lidar_pcs in this log files is {result['cnt']}.")

2021.10.21.14.43.30_veh-28_01244_01519.db
The number of lidar_pcs in this log files is 5500.
